# SPACY

Load JD


## Match the whole text for example skills

In [31]:
import os 
import json
from collections import defaultdict
import PyPDF2
#import pandas as pd 
#import numpy as np
import matplotlib.pyplot as plt
import spacy 
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example
from tqdm import tqdm
import re
from tika import parser
import warnings
warnings.filterwarnings("ignore")

In [11]:
def read_jd_file(filename):
    """Program to read the entire file (absolute path) using read() function"""
    file = open(filename, "r")
    content = file.read()
    file.close()
    return content

def preprocess(text):
  text = "".join([s for s in text.splitlines(True) if s.strip("\r\n")])
  # text = re.sub('[^A-Za-z0-9\n]+', ' ', text)
  return text

sub_directory_for_jd = 'JD'
files_list = os.listdir('JD')


In [20]:
complete_file_paths = [os.path.join(sub_directory_for_jd, file) for file in files_list]
complete_file_paths

jd_text = read_jd_file(complete_file_paths[1])
jd_text = preprocess(jd_text)


### Load Model for NER

This model has to be fine tune using GPU training again with resume/jobdescription dataset


In [6]:
nlp = spacy.load('./model')

### Extract NER labels from the models

In [21]:
def extract_ner(text, nlp):
    '''This function get the conntent of resume and label them '''
    ner_labels = []
    for doc in nlp.pipe([text], disable=["tagger", "parser"]):
      for ent in doc.ents:
          text_name = re.sub('[^A-Za-z0-9]+', ' ', ent.text).strip()
          ner_labels.append((text_name, ent.label_))
    return ner_labels

ner_labels = extract_ner(jd_text, nlp)

In [22]:
def resume_json(ner_labels):
    # Create a defaultdict to group items by label
    grouped_data = defaultdict(list)
    
    # Group the items by their labels
    for item, label in ner_labels:
        grouped_data[label].append(item)
    
    # Convert the defaultdict to a regular dictionary
    json_data = dict(grouped_data)
    
    # Convert the dictionary to a JSON string
    json_string = json.dumps(json_data, indent=2)
    return json_string

jd_json = resume_json(ner_labels)
print(jd_json)

{
  "TOOL": [
    "Oracle DBA",
    "Oracle EBS",
    "AP FA GL and Coupa",
    "Oracle database",
    "EBS R12 2 x objects",
    "Oracle team",
    "Oracle SRS",
    "SQL",
    "Pl sql",
    "Putty",
    "WinSCP",
    "Joms"
  ],
  "EXPERIENCE": [
    "6 months",
    "Previous banking knowledge"
  ]
}


In [25]:
def extract_skills(ner_results, label):
    '''This function extracts skills from NER results
    label can be SKILL, EXPERIENCE, TOOL ...'''
    
    results = [item[0] for item in ner_results if item[1] == label]
    return results

# Extract skills
skills_jd = extract_skills(ner_labels, 'SKILL')
experience_jd = extract_skills(ner_labels, 'EXPERIENCE')
tool_jd = extract_skills(ner_labels, 'TOOL')

print(tool_jd)

['Oracle DBA', 'Oracle EBS', 'AP FA GL and Coupa', 'Oracle database', 'EBS R12 2 x objects', 'Oracle team', 'Oracle SRS', 'SQL', 'Pl sql', 'Putty', 'WinSCP', 'Joms']


## CHECK file from spider

In [26]:
def load_model():
  nlp = spacy.load('./model')
  return nlp

In [27]:
def load_json_file(file_number):
  '''Loads text data of training data which is in annotated form. Enter file_number between 0 to 50 '''
  spider_tagged_data = 'spider_software_tagged_data'
  list_tagged_files = os.listdir(spider_tagged_data)
  file_path = os.path.join(spider_tagged_data,list_tagged_files[file_number])
  with open(file_path,'r') as f:
    data = json.load(f)
  return data['annotations']


In [32]:
# Taken from https://stackoverflow.com/questions/68213223/how-to-evaluate-trained-spacy-version-3-model#:~:text=nlp%20%3D%20spacy.load(path_to_model)%0Aexamples%20%3D%20%5B%5D%0Ascorer%20%3D%20Scorer()%0Afor%20text%2C%20annotations%20in%20TEST_REVISION_DATA%3A%0A%20%20%20%20doc%20%3D%20nlp.make_doc(text)%0A%20%20%20%20example%20%3D%20Example.from_dict(doc%2C%20annotations)%0A%20%20%20%20example.predicted%20%3D%20nlp(str(example.predicted))%0A%20%20%20%20examples.append(example)%0Ascorer.score(examples)
def score_metrics(nlp,data):
  examples = []
  scorer = Scorer()
  for text, annotations in data:
      doc = nlp.make_doc(text)
      example = Example.from_dict(doc, annotations)
      example.predicted = nlp(str(example.predicted))
      examples.append(example)
  result_metrics = scorer.score(examples)
  return result_metrics['ents_per_type']

In [39]:
def function_2(file_number):
    '''In this function it takes file number as input and correspondingly choose required annotated file from the folder.And after that it can 
    show the output of precision,recall,f1_score
    '''
    data = load_json_file(file_number)
    print('The data looks like:')
    print('='*100)
    for a in data:
        print(a)
    nlp = load_model()
    
    result_net = score_metrics(nlp,data)
    print('='*100)
    print('The result metrics are down below')
    print('-'*100)
    for i,j in result_net.items():
       print(i,' ',j)

In [40]:
# Here you can choose any number between 0 to 50 including both of them
function_2(0)

The data looks like:
['Network Engineer Network Admin Resume', {'entities': [[0, 16, 'JOB_TITLE'], [17, 30, 'JOB_TITLE']]}]
['Job Level Experienced with over 2 years experience', {'entities': [[10, 50, 'EXPERIENCE']]}]
['Objective SUMMARYAn IT professional with over 10 years of experience in engineering administration implementation network monitoring and management analysis escalation support documentation configuration and troubleshooting for medium to global enterprise environments which includes proficiency in routing routing protocols switching security voice wireless data center technologies and firewall management ', {'entities': [[20, 83, 'EXPERIENCE'], [84, 407, 'EXPERIENCE']]}]
['Experience Spydaweb LLC Consultant 2013 Present Responsibilities include management of various teams to facilitate the creation of new Consumer product invention Management of vendor relationships and triage of associated technology challenges were a key part of this role Managed Product design and e